In [203]:
#%cd '/Users/pc/Desktop/Deep Learning/Project Github/CS7643_final_project'

In [204]:
#!python src/holdup/datapickler.py

## This is a notebook that creates two dictionaries: d_nodes will provide the accuracy of the preflop, flop, turn and river data set for different hidden nodes and d_epochs will provide the accuracy for different epochs. 

## The code takes alot of time to run. For that reason I have already saved the dictionaries into the following directory: src/holdup/visualizations/json_dictionaries. This notebook should only be used in case somebody wants to regenerate the dictionaries.

In [191]:
from holdup.the_model.autoencoder import Autoencoder
import numpy as np
# import pandas as pd
import torch
from torch import nn
import torch.optim as optim
# from sklearn.model_selection import train_test_split
import random
import os
from holdup.parser.replayable_hand import ReplayableHand, Streets
import functools
from typing import Tuple, List
import matplotlib.pyplot as plt
from holdup.the_model.get_datasets import *
import pandas as pd
from sklearn.model_selection import train_test_split

preflop = "preflop"
flop = "flop"
turn = "turn"
river = "river"

def get_stage(dataset, stage):
    if stage == preflop:
        return dataset[0]
    if stage == flop:
        return dataset[1]
    if stage == turn:
        return dataset[2]
    if stage == river:
        return dataset[3]

def flatten_streets(dataset):
    streets = [[], [], [], []]
    for logfile in dataset:
        for index, street in enumerate(logfile):
            streets[index] = streets[index] + street
    return streets


def get_data(dataset, stage):
    flattened_data = flatten_streets(dataset)
    stage_data = get_stage(flattened_data, stage)
    return [(x[0], x[1][1]) for x in stage_data]


with open('last_possible.pickle', 'rb') as last_possible_pickle:
    last_possible_dataset = pickle.load(last_possible_pickle)

preflop_data = get_data(last_possible_dataset, "preflop")
flop_data = get_data(last_possible_dataset, "flop")
turn_data = get_data(last_possible_dataset, "turn")
river_data = get_data(last_possible_dataset, "river")

def separate_train_test(street_data):
    n_train = int(len(street_data)*0.6)
    train_set = street_data[:n_train]
    test_set = street_data[n_train:]
    return train_set,test_set

train_preflop, test_preflop =separate_train_test(preflop_data)
print("preflop_train_data_size: {}".format(len(train_preflop)))
print("preflop_test_data_size: {}".format(len(test_preflop)))

train_flop, test_flop=separate_train_test(flop_data)
print("flop_train_data_size: {}".format(len(train_flop)))
print("flop_test_data_size: {}".format(len(test_flop)))

train_turn, test_turn=separate_train_test(turn_data)
print("turn_train_data_size: {}".format(len(train_turn)))
print("turn_test_data_size: {}".format(len(test_turn)))

train_river, test_river=separate_train_test(river_data)
print("river_train_data_size: {}".format(len(train_river)))
print("river_test_data_size: {}".format(len(test_river)))


# Set the device to use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create an instance of the autoencoder
# model = Autoencoder(num_hidden_nodes).to(device)

def train2(learning_rate, model, train_loader, num_epochs, wd, pftr='stage_name'):
    criterion = nn.CrossEntropyLoss() #changed to cross entropy loss for classification based tasks (semi-supervised)
    optimizer = optim.Adam(model.parameters(),lr=learning_rate, weight_decay=wd)
    train_losses = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        for data in train_loader:
            inputs, labels = data
            inputs = inputs.float()
            optimizer.zero_grad()
            batch_size, _, _ = inputs.size()
            inputs = inputs.view(batch_size, -1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)


    
def quick_test2(model,test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.float()
            batch_size, _, _ = inputs.size()
            inputs = inputs.view(batch_size, -1)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return int(round((correct/total)*100, 0))


def train_and_quick_test(lr, num_hidden_nodes, num_epochs, weight_decay,train_data,test_data, pftr):
    # Define the model
    model = Autoencoder(num_hidden_nodes).to(device)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=20, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=20, shuffle=False)
    train2(lr, model, train_loader, num_epochs, weight_decay,pftr)
    # Test the model
    quick_test(model,test_loader)
    

###
def get_visualization_parameters_nodes(lr, weight_decay,train_preflop, train_flop, train_turn, train_river, test_preflop, test_flop, test_turn, test_river, pftr_preflop, pftr_flop, pftr_turn, pftr_river):
    final_dict = {}
    for i in range(10,110,10):
        
        final_dict[str(i)] = []
        
        model = Autoencoder(i).to(device)
        train_loader = torch.utils.data.DataLoader(train_preflop, batch_size=20, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_preflop, batch_size=20, shuffle=False)
        train2(lr, model, train_loader, 20, weight_decay,pftr_preflop)
        result = quick_test2(model,test_loader)
        final_dict[str(i)].append(result)

        model = Autoencoder(i).to(device)
        train_loader = torch.utils.data.DataLoader(train_flop, batch_size=20, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_flop, batch_size=20, shuffle=False)
        train2(lr, model, train_loader, 20, weight_decay,pftr_flop)
        result = quick_test2(model,test_loader)
        final_dict[str(i)].append(result)
        

        
        model = Autoencoder(i).to(device)
        train_loader = torch.utils.data.DataLoader(train_turn, batch_size=20, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_turn, batch_size=20, shuffle=False)
        train2(lr, model, train_loader, 40, weight_decay,pftr_turn)
        result = quick_test2(model,test_loader)
        final_dict[str(i)].append(result)
        
        model = Autoencoder(i).to(device)
        train_loader = torch.utils.data.DataLoader(train_river, batch_size=20, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_river, batch_size=20, shuffle=False)
        train2(lr, model, train_loader, 40, weight_decay,pftr_river)
        result = quick_test2(model,test_loader)
        final_dict[str(i)].append(result)
        
        final_dict[str(i)] = tuple(final_dict[str(i)])
        
        print(final_dict)

        
    
    return final_dict


def get_visualization_parameters_epochs(lr, weight_decay,train_preflop, train_flop, train_turn, train_river, test_preflop, test_flop, test_turn, test_river, pftr_preflop, pftr_flop, pftr_turn, pftr_river):

    final_dict = {}
    for i in range(10,70,10):
        
        final_dict[str(i)] = []
        
        model = Autoencoder(20).to(device)
        train_loader = torch.utils.data.DataLoader(train_preflop, batch_size=20, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_preflop, batch_size=20, shuffle=False)
        train2(lr, model, train_loader, i, weight_decay,pftr_preflop)
        result = quick_test2(model,test_loader)
        final_dict[str(i)].append(result)

        model = Autoencoder(20).to(device)
        train_loader = torch.utils.data.DataLoader(train_flop, batch_size=20, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_flop, batch_size=20, shuffle=False)
        train2(lr, model, train_loader, i, weight_decay,pftr_flop)
        result = quick_test2(model,test_loader)
        final_dict[str(i)].append(result)
        
        model = Autoencoder(40).to(device)
        train_loader = torch.utils.data.DataLoader(train_turn, batch_size=20, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_turn, batch_size=20, shuffle=False)
        train2(lr, model, train_loader, i, weight_decay,pftr_turn)
        result = quick_test2(model,test_loader)
        final_dict[str(i)].append(result)
        
        model = Autoencoder(40).to(device)
        train_loader = torch.utils.data.DataLoader(train_river, batch_size=20, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_river, batch_size=20, shuffle=False)
        train2(lr, model, train_loader, i, weight_decay,pftr_river)
        result = quick_test2(model,test_loader)
        final_dict[str(i)].append(result)
        
        final_dict[str(i)] = tuple(final_dict[str(i)])
        
        print(final_dict)
        
    
    return final_dict

def get_visualization_dictionaries(lr, weight_decay,train_preflop, train_flop, train_turn, train_river, test_preflop, test_flop, test_turn, test_river, pftr_preflop, pftr_flop, pftr_turn, pftr_river):
    
    d_f1 = get_visualization_parameters_nodes(lr, weight_decay,train_preflop, train_flop, train_turn, train_river, test_preflop, test_flop, test_turn, test_river, pftr_preflop, pftr_flop, pftr_turn, pftr_river)
    d_f2 = get_visualization_parameters_epochs(lr, weight_decay,train_preflop, train_flop, train_turn, train_river, test_preflop, test_flop, test_turn, test_river, pftr_preflop, pftr_flop, pftr_turn, pftr_river)
        
    return d_f1, d_f2




preflop_train_data_size: 30650
preflop_test_data_size: 20434
flop_train_data_size: 23450
flop_test_data_size: 15634
turn_train_data_size: 18447
turn_test_data_size: 12298
river_train_data_size: 16371
river_test_data_size: 10915


# Tuning the model

## Best Model Parameters: 

### Learning Rate: .01 
### Weight Decay: 0 (as it decreases it improves test accuracy)
### Batch Size: 20 (Using a batch size of 20 as default as batch size doesn't have significant impact on model performance)

## Code below is used to see the test accuracy obtained by using different learning rates with different weight decays. It can be seen that using a learning rate of .01 gave the best performance out of all of the models and that by decreasing weight decay the model works better and for that reason we opted to using a weight decay of zero.

In [169]:


def tune_model(learning_rate, weight_decay, num_epochs, train_preflop, test_preflop, pftr, hidden_nodes, batch_size):
        lr = {}
        for i in range(len(learning_rate)):
            lr[str(learning_rate[i])] = []
            for j in range(len(weight_decay)):
                model = Autoencoder(hidden_nodes).to(device)
                train_loader = torch.utils.data.DataLoader(train_flop, batch_size=batch_size, shuffle=True)
                test_loader = torch.utils.data.DataLoader(test_flop, batch_size=batch_size, shuffle=False)
                train2(learning_rate[i], model, train_loader, num_epochs, weight_decay[j],pftr)
                result = quick_test2(model,test_loader)
                lr[str(learning_rate[i])].append(result)
        return lr
    
    
    
    
    

In [172]:
lr = [.0001, .001, .01, .1]
wd = [0, .0001, .1]

pre_flop = tune_model(lr, wd, 20, train_preflop, test_preflop, pftr='preflop_last_possible', hidden_nodes=20, batch_size=20)


In [173]:
pre_flop

print("PRE FLOP")
for key, value in pre_flop.items():
    print("learning rate: {}".format(key))
    for j in range(len(value)):
        print("     weight decay: {}".format(wd[j]), "accuracy: {}".format(value[j]))
    

PRE FLOP
learning rate: 0.0001
     weight decay: 0 accuracy: 83
     weight decay: 0.0001 accuracy: 83
     weight decay: 0.1 accuracy: 58
learning rate: 0.001
     weight decay: 0 accuracy: 85
     weight decay: 0.0001 accuracy: 84
     weight decay: 0.1 accuracy: 58
learning rate: 0.01
     weight decay: 0 accuracy: 86
     weight decay: 0.0001 accuracy: 84
     weight decay: 0.1 accuracy: 58
learning rate: 0.1
     weight decay: 0 accuracy: 82
     weight decay: 0.0001 accuracy: 77
     weight decay: 0.1 accuracy: 58


In [174]:
lr = [.0001, .001, .01, .1]
wd = [0, .0001, .1]

flop = tune_model(lr, wd, 20, train_flop, test_flop, pftr='flop_last_possible', hidden_nodes=20, batch_size=20)

In [175]:
print("FLOP")
for key, value in flop.items():
    print("learning rate: {}".format(key))
    for j in range(len(value)):
        print("     weight decay: {}".format(wd[j]), "accuracy: {}".format(value[j]))

FLOP
learning rate: 0.0001
     weight decay: 0 accuracy: 83
     weight decay: 0.0001 accuracy: 83
     weight decay: 0.1 accuracy: 58
learning rate: 0.001
     weight decay: 0 accuracy: 84
     weight decay: 0.0001 accuracy: 84
     weight decay: 0.1 accuracy: 58
learning rate: 0.01
     weight decay: 0 accuracy: 86
     weight decay: 0.0001 accuracy: 84
     weight decay: 0.1 accuracy: 58
learning rate: 0.1
     weight decay: 0 accuracy: 84
     weight decay: 0.0001 accuracy: 77
     weight decay: 0.1 accuracy: 58


In [176]:
lr = [.0001, .001, .01, .1]
wd = [0, .0001, .1]

turn = tune_model(lr, wd, 40, train_turn, test_turn, pftr='turn_last_possible', hidden_nodes=40, batch_size=20)

In [177]:
print("TURN")
for key, value in turn.items():
    print("learning rate: {}".format(key))
    for j in range(len(value)):
        print("     weight decay: {}".format(wd[j]), "accuracy: {}".format(value[j]))

TURN
learning rate: 0.0001
     weight decay: 0 accuracy: 83
     weight decay: 0.0001 accuracy: 83
     weight decay: 0.1 accuracy: 58
learning rate: 0.001
     weight decay: 0 accuracy: 87
     weight decay: 0.0001 accuracy: 85
     weight decay: 0.1 accuracy: 58
learning rate: 0.01
     weight decay: 0 accuracy: 87
     weight decay: 0.0001 accuracy: 84
     weight decay: 0.1 accuracy: 58
learning rate: 0.1
     weight decay: 0 accuracy: 83
     weight decay: 0.0001 accuracy: 80
     weight decay: 0.1 accuracy: 58


In [178]:
lr = [.0001, .001, .01, .1]
wd = [0, .0001, .1]

river = tune_model(lr, wd, 40, train_river, test_river, pftr='river_last_possible', hidden_nodes=40, batch_size=20)

In [179]:
print("RIVER")
for key, value in river.items():
    print("learning rate: {}".format(key))
    for j in range(len(value)):
        print("     weight decay: {}".format(wd[j]), "accuracy: {}".format(value[j]))

RIVER
learning rate: 0.0001
     weight decay: 0 accuracy: 83
     weight decay: 0.0001 accuracy: 83
     weight decay: 0.1 accuracy: 58
learning rate: 0.001
     weight decay: 0 accuracy: 87
     weight decay: 0.0001 accuracy: 84
     weight decay: 0.1 accuracy: 58
learning rate: 0.01
     weight decay: 0 accuracy: 87
     weight decay: 0.0001 accuracy: 85
     weight decay: 0.1 accuracy: 58
learning rate: 0.1
     weight decay: 0 accuracy: 83
     weight decay: 0.0001 accuracy: 81
     weight decay: 0.1 accuracy: 58


## Code Below is used to demonstrate how changing the batch size does not make significant change to improving the test accuracy of the model


In [197]:
lr = [.01]
wd = [0]
b_s = [8,16,32,64,128,256,512,1024]

ls = []

for z in range(len(b_s)):
    pre_flop = tune_model(lr, wd, 20, train_preflop, test_preflop, pftr='preflop_last_possible', hidden_nodes=20, batch_size=b_s[z])
    ls.append(pre_flop)

print("PRE FLOP")
print("Using learning rate of {} and weight decay of {}:".format(lr[0], wd[0]))
for i in range(len(ls)):
    for key, value in ls[i].items():
        print("     When using batch size of {}, accuracy is {}".format(b_s[i], value[0]))



PRE FLOP
Using learning rate of 0.01 and weight decay of 0:
     When using batch size of 8, accuracy is 86
     When using batch size of 16, accuracy is 87
     When using batch size of 32, accuracy is 87
     When using batch size of 64, accuracy is 87
     When using batch size of 128, accuracy is 86
     When using batch size of 256, accuracy is 86
     When using batch size of 512, accuracy is 85
     When using batch size of 1024, accuracy is 84


In [202]:
lr = [.01]
wd = [0]
b_s = [8,16,32,64,128,256,512,1024]

ls = []

for z in range(len(b_s)):
    flop = tune_model(lr, wd, 20, train_flop, test_flop, pftr='flop_last_possible', hidden_nodes=20, batch_size=b_s[z])
    ls.append(flop)

print("FLOP")
print("Using learning rate of {} and weight decay of {}:".format(lr[0], wd[0]))
for i in range(len(ls)):
    for key, value in ls[i].items():
        print("     When using batch size of {}, accuracy is {}".format(b_s[i], value[0]))


FLOP
Using learning rate of 0.01 and weight decay of 0:
     When using batch size of 8, accuracy is 86
     When using batch size of 16, accuracy is 87
     When using batch size of 32, accuracy is 87
     When using batch size of 64, accuracy is 86
     When using batch size of 128, accuracy is 87
     When using batch size of 256, accuracy is 86
     When using batch size of 512, accuracy is 86
     When using batch size of 1024, accuracy is 84


In [200]:
lr = [.01]
wd = [0]
b_s = [8,16,32,64,128,256,512,1024]

ls = []

for z in range(len(b_s)):
    turn = tune_model(lr, wd, 40, train_turn, test_turn, pftr='turn_last_possible', hidden_nodes=40, batch_size=b_s[z])
    ls.append(turn)
    
print("TURN")
print("Using learning rate of {} and weight decay of {}:".format(lr[0], wd[0]))
for i in range(len(ls)):
    for key, value in ls[i].items():
        print("     When using batch size of {}, accuracy is {}".format(b_s[i], value[0]))


TURN
Using learning rate of 0.01 and weight decay of 0:
     When using batch size of 8, accuracy is 86
     When using batch size of 16, accuracy is 87
     When using batch size of 32, accuracy is 87
     When using batch size of 64, accuracy is 87
     When using batch size of 128, accuracy is 87
     When using batch size of 256, accuracy is 87
     When using batch size of 512, accuracy is 87
     When using batch size of 1024, accuracy is 87


In [201]:
lr = [.01]
wd = [0]
b_s = [8,16,32,64,128,256,512,1024]

ls = []

for z in range(len(b_s)):
    river = tune_model(lr, wd, 40, train_river, test_river, pftr='river_last_possible', hidden_nodes=40, batch_size=b_s[z])
    ls.append(river)
    
print("RIVER")
print("Using learning rate of {} and weight decay of {}:".format(lr[0], wd[0]))
for i in range(len(ls)):
    for key, value in ls[i].items():
        print("     When using batch size of {}, accuracy is {}".format(b_s[i], value[0]))


RIVER
Using learning rate of 0.01 and weight decay of 0:
     When using batch size of 8, accuracy is 87
     When using batch size of 16, accuracy is 87
     When using batch size of 32, accuracy is 87
     When using batch size of 64, accuracy is 87
     When using batch size of 128, accuracy is 87
     When using batch size of 256, accuracy is 87
     When using batch size of 512, accuracy is 87
     When using batch size of 1024, accuracy is 86


## Running the code line below will generate two dictionaries: d_nodes and d_epochs 

In [193]:
d_nodes2, d_epochs2 = get_visualization_dictionaries(.01, 0,train_preflop, train_flop, train_turn, train_river, test_preflop, test_flop, test_turn, test_river, pftr_preflop='preflop_last_possible', pftr_flop='flop_last_possible', pftr_turn='turn_last_possible', pftr_river='river_last_possible')

{'10': (83, 86, 91, 89)}
{'10': (83, 86, 91, 89), '20': (83, 86, 92, 89)}
{'10': (83, 86, 91, 89), '20': (83, 86, 92, 89), '30': (83, 87, 92, 89)}
{'10': (83, 86, 91, 89), '20': (83, 86, 92, 89), '30': (83, 87, 92, 89), '40': (83, 87, 92, 90)}
{'10': (83, 86, 91, 89), '20': (83, 86, 92, 89), '30': (83, 87, 92, 89), '40': (83, 87, 92, 90), '50': (83, 87, 92, 90)}
{'10': (83, 86, 91, 89), '20': (83, 86, 92, 89), '30': (83, 87, 92, 89), '40': (83, 87, 92, 90), '50': (83, 87, 92, 90), '60': (83, 87, 92, 89)}
{'10': (83, 86, 91, 89), '20': (83, 86, 92, 89), '30': (83, 87, 92, 89), '40': (83, 87, 92, 90), '50': (83, 87, 92, 90), '60': (83, 87, 92, 89), '70': (83, 87, 92, 90)}
{'10': (83, 86, 91, 89), '20': (83, 86, 92, 89), '30': (83, 87, 92, 89), '40': (83, 87, 92, 90), '50': (83, 87, 92, 90), '60': (83, 87, 92, 89), '70': (83, 87, 92, 90), '80': (83, 87, 92, 90)}
{'10': (83, 86, 91, 89), '20': (83, 86, 92, 89), '30': (83, 87, 92, 89), '40': (83, 87, 92, 90), '50': (83, 87, 92, 90), '60': (

In [194]:
d_nodes2

{'10': (83, 86, 91, 89),
 '20': (83, 86, 92, 89),
 '30': (83, 87, 92, 89),
 '40': (83, 87, 92, 90),
 '50': (83, 87, 92, 90),
 '60': (83, 87, 92, 89),
 '70': (83, 87, 92, 90),
 '80': (83, 87, 92, 90),
 '90': (84, 87, 92, 90),
 '100': (83, 87, 92, 90)}

In [195]:
d_epochs2

{'10': (83, 86, 92, 90),
 '20': (83, 86, 92, 90),
 '30': (83, 86, 93, 90),
 '40': (83, 87, 92, 90),
 '50': (83, 86, 92, 90),
 '60': (83, 87, 92, 90)}

## Running the line below will export the dictionaires as json files to src/holdup/visualizations/json_dictionaries

In [196]:
import json

with open(r'src/holdup/visualizations/json_dictionaries/d_nodes.json', 'w') as f:
    json.dump(d_nodes2, f)

with open(r'src/holdup/visualizations/json_dictionaries/d_epochs.json', 'w') as f:
    json.dump(d_epochs2, f)